<a href="https://colab.research.google.com/github/ach224/Prediction_eligibilite_pret_bancaire/blob/Modelisation_A%C3%AFcha/Modelisation_Aicha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etape 3 : Modélisation

* Définir variable cible : Loan_Status
* Séparer train/test
* Tester plusieurs modèles (Logistic Regression, Decision Tree, Random Forest, Gradient Boosting)
* Comparer performances (Accuracy, Precision, Recall, F1-score, ROC-AUC)

In [6]:
# Récupération du dernier dataframe
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/MyDrive/DATA SCIENCES/PROJET DATA SCIENCES/loan_prediction.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
# Traitement des valeurs manquantes
  # Colonnes numériques : remplacer par la médiane
for col in ['LoanAmount', 'Loan_Amount_Term']:
    df[col] = df[col].fillna(df[col].median())

  # Colonnes categorielle : remplacer par le mode
for col in ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']:
    df[col] = df[col].fillna(df[col].mode()[0])

# Suppression de la colonne Loan_ID
df.drop('Loan_ID', axis=1, inplace=True)

# Encodage des variables
# Colonnes catégorielles à encoder
cat_cols = ['Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area']

# One-Hot Encoding
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Convertir la variable df['Dependents'] en int
df['Dependents'] = df['Dependents'].replace('3+', 3).astype(int)
df['CoapplicantIncome'] = df['CoapplicantIncome'].astype(int)
df['LoanAmount'] = df['LoanAmount'].astype(int)
df['Loan_Amount_Term'] = df['Loan_Amount_Term'].astype(int)
df['Credit_History'] = df['Credit_History'].astype(int)

# Encoder Loan_Status
df['Loan_Status'] = df['Loan_Status'].map({'Y': 1, 'N': 0})

# Vérification
print(df['Loan_Status'].value_counts())
print(df.info())
df.head()

Loan_Status
1    422
0    192
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype
---  ------                   --------------  -----
 0   Dependents               614 non-null    int64
 1   ApplicantIncome          614 non-null    int64
 2   CoapplicantIncome        614 non-null    int64
 3   LoanAmount               614 non-null    int64
 4   Loan_Amount_Term         614 non-null    int64
 5   Credit_History           614 non-null    int64
 6   Loan_Status              614 non-null    int64
 7   Gender_Male              614 non-null    bool 
 8   Married_Yes              614 non-null    bool 
 9   Education_Not Graduate   614 non-null    bool 
 10  Self_Employed_Yes        614 non-null    bool 
 11  Property_Area_Semiurban  614 non-null    bool 
 12  Property_Area_Urban      614 non-null    bool 
dtypes: bool(6), int64(7)
memory usage: 37.3 KB
None


,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,0,5849,0,128,360,1,1,True,False,False,False,False,True
1,1,4583,1508,128,360,1,0,True,True,False,False,False,False
2,0,3000,0,66,360,1,1,True,True,False,True,False,True
3,0,2583,2358,120,360,1,1,True,True,True,False,False,True
4,0,6000,0,141,360,1,1,True,False,False,False,False,True


## 1. Définir la variable cible

Nous savons que notre variable cible est ['Loan_Status'].


In [13]:
y = df['Loan_Status']
X = df.drop(columns = ['Loan_Status'])

print("Shape de X:", X.shape)
print("Shape de y:", y.shape)

X.head()

Shape de X: (614, 12)
Shape de y: (614,)


,Dependents,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Gender_Male,Married_Yes,Education_Not Graduate,Self_Employed_Yes,Property_Area_Semiurban,Property_Area_Urban
0,0,5849,0,128,360,1,True,False,False,False,False,True
1,1,4583,1508,128,360,1,True,True,False,False,False,False
2,0,3000,0,66,360,1,True,True,False,True,False,True
3,0,2583,2358,120,360,1,True,True,True,False,False,True
4,0,6000,0,141,360,1,True,False,False,False,False,True
